SMB、HMI两个指标的计算方式如下：
1. 首先，按市值大小平均分为两组(Small 组, Big 组)，基准是这一时间的市场上公司市值中位数;
2. 按 BM 从小到大分三组,即前 30%(Growth 组),中间 40%(Neutral 组),后 30%(Value 组)；
3. 每个组的月回报以组内所有成员股票当月回报的加权平均数为依据,计算每个月的 SMB 和 HML 值。具体计算公式如下: 
   SMB = Small size return - Big size return)
   HML = Value company return - Growth company return
4. 以市场回报减去无风险回报,即得到超额市场回报(Rm-Rf)。（由于Ricequant平台无法提取国债收益率，我只能对这个时段的所有IBO1M求平均值作为无风    险回报率

In [10]:
%pdb

Automatic pdb calling has been turned OFF


In [11]:
# %load fama.py
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_book_value=pd.read_csv("./book_value_per_share.csv",index_col=0)
df_market_cap=pd.read_csv("./market_cap.csv",index_col=0)
price_data=pd.read_csv("./price_data.csv",index_col=0)

index_date=df_book_value.index
stocks=df_book_value.columns
format='%Y-%m-%d %H:%M:%S'


In [12]:
df_book_value

,002334.XSHE,000683.XSHE,002326.XSHE,600841.XSHG,000541.XSHE,600520.XSHG,600382.XSHG,600208.XSHG,002168.XSHE,600895.XSHG,...,600422.XSHG,002193.XSHE,600735.XSHG,600179.XSHG,600695.XSHG,600657.XSHG,600531.XSHG,002343.XSHE,000530.XSHE,600491.XSHG
2010-11-01,7.93,2.7000,6.49,4.0700,2.7200,1.8760,2.6800,1.3800,1.8000,3.7600,...,2.2590,4.11,1.1228,1.3200,0.2500,3.5400,4.7800,10.92,4.9600,5.0480
2010-09-30,14.78,2.6800,6.40,3.9900,2.5800,1.8569,2.6620,1.2630,1.7302,3.6600,...,2.1662,4.04,1.5532,1.3800,0.2250,3.4820,4.5580,10.68,4.8500,5.0010
2010-08-30,14.78,2.6800,6.40,3.9900,2.5800,1.8569,2.6620,1.2630,1.7302,3.6600,...,2.1662,4.04,1.5532,1.3800,0.2500,3.4820,4.5580,10.68,4.8500,5.0010
2010-07-30,14.34,4.0100,6.56,4.0100,2.7500,1.8569,2.6600,1.8600,1.6600,3.9400,...,2.1600,4.03,1.4154,1.5500,0.2500,3.4500,4.5580,10.68,4.8800,4.8900
2010-06-30,14.34,4.0100,6.56,4.0100,2.7500,1.8340,2.6600,1.8600,1.6600,3.9400,...,2.1600,4.03,1.4154,1.5500,0.2500,3.4500,4.7100,10.37,4.8800,4.8900
2010-05-28,14.34,4.0100,6.56,4.0100,2.7500,1.8340,2.6600,1.8600,1.6600,3.9400,...,2.1600,4.03,1.4154,1.5500,0.2500,3.4500,4.7100,10.37,4.8800,4.8900
2010-04-28,14.34,3.9800,6.56,4.0100,2.6400,2.1000,2.6600,1.8600,1.6600,3.7400,...,2.1600,4.03,1.2800,1.5800,0.2500,3.4300,4.7100,10.37,4.8800,4.8000
2010-03-26,4.17,2.4800,6.47,3.9500,2.6400,2.1000,2.6300,1.8900,2.6800,3.7400,...,2.0600,3.86,1.2800,1.9900,0.3190,3.4300,4.3800,4.72,4.8600,4.5900
2010-02-26,3.43,2.4800,2.24,3.8600,2.6400,2.1000,2.4200,1.8900,2.4900,3.5600,...,2.0600,3.86,1.2800,1.9900,0.3190,3.0300,4.3800,4.46,4.8600,4.5900
2010-01-26,3.43,2.4800,2.24,3.8600,2.6400,2.1000,2.4200,1.7600,2.4900,3.5600,...,2.0600,3.86,1.2800,1.9900,0.3190,3.0300,4.3800,4.46,4.8600,4.5900


In [13]:
df_market_cap

,002334.XSHE,000683.XSHE,002326.XSHE,600841.XSHG,000541.XSHE,600520.XSHG,600382.XSHG,600208.XSHG,002168.XSHE,600895.XSHG,...,600422.XSHG,002193.XSHE,600735.XSHG,600179.XSHG,600695.XSHG,600657.XSHG,600531.XSHG,002343.XSHE,000530.XSHE,600491.XSHG
2010-11-01,7.753216e+09,8.829861e+09,7.509375e+09,7.800223e+09,2.015841e+10,1.723860e+09,3.437971e+09,3.203337e+10,8.024718e+09,1.604442e+10,...,4.367046e+09,2.457600e+09,2.286432e+09,2.028000e+09,4.084886e+09,1.079176e+10,7.697236e+09,3.116428e+09,4.515193e+09,6.784816e+09
2010-09-30,6.563968e+09,7.616715e+09,7.524060e+09,7.444794e+09,1.682151e+10,1.674122e+09,2.976613e+09,2.487536e+10,5.918643e+09,1.513070e+10,...,4.345054e+09,2.489600e+09,2.627098e+09,2.028000e+09,3.996967e+09,1.018206e+10,4.811914e+09,2.978734e+09,4.116176e+09,4.922782e+09
2010-08-30,6.714240e+09,7.117636e+09,8.010000e+09,8.049984e+09,1.786857e+10,1.766815e+09,2.778400e+09,2.700753e+10,5.882590e+09,1.527008e+10,...,4.056012e+09,2.468800e+09,2.232093e+09,2.028000e+09,4.193095e+09,1.105089e+10,3.953622e+09,3.175864e+09,3.871166e+09,5.410796e+09
2010-07-30,5.224960e+09,6.119477e+09,6.582885e+09,7.291095e+09,1.320082e+10,1.745338e+09,2.498168e+09,2.812438e+10,4.611734e+09,1.596699e+10,...,3.748120e+09,2.521600e+09,2.023095e+09,2.028000e+09,3.841416e+09,1.182826e+10,3.622632e+09,3.011424e+09,3.416146e+09,4.927520e+09
2010-06-30,5.120000e+09,4.699022e+09,6.421350e+09,6.244021e+09,1.162534e+10,1.442390e+09,2.211101e+09,2.421540e+10,4.137041e+09,1.339616e+10,...,3.207737e+09,2.361600e+09,1.921229e+09,1.723800e+09,3.205689e+09,9.724782e+09,3.191203e+09,2.714244e+09,2.702116e+09,3.709854e+09
2010-05-28,6.126080e+09,6.372856e+09,7.027440e+09,7.430385e+09,1.448274e+10,1.537344e+09,2.792070e+09,2.512919e+10,4.542633e+09,1.537849e+10,...,3.839231e+09,2.515200e+09,2.393408e+09,2.004600e+09,4.030782e+09,1.059361e+10,3.914816e+09,2.781605e+09,3.762661e+09,4.946472e+09
2010-04-28,5.112320e+09,8.604635e+09,6.801825e+09,8.784857e+09,1.401303e+10,1.798466e+09,3.161156e+09,2.604298e+10,5.179564e+09,1.940508e+10,...,4.260227e+09,2.155200e+09,2.898098e+09,2.172300e+09,4.835586e+09,1.163011e+10,4.759412e+09,3.379927e+09,3.895667e+09,5.600316e+09
2010-03-26,4.480640e+09,7.693496e+09,5.341335e+09,8.386200e+09,1.232990e+10,1.631167e+09,3.335447e+09,3.137341e+10,4.887137e+09,1.841392e+10,...,3.700993e+09,2.368000e+09,2.743285e+09,2.710500e+09,5.525418e+09,1.454144e+10,4.143085e+09,3.622624e+09,4.308684e+09,7.850866e+09
2010-02-26,4.510080e+09,8.056928e+09,5.202495e+09,8.669583e+09,1.086206e+10,1.500041e+09,3.014205e+09,3.100113e+10,4.476537e+09,1.830551e+10,...,3.996319e+09,2.489600e+09,2.799018e+09,2.808000e+09,5.295474e+09,1.466339e+10,4.275481e+09,3.774186e+09,3.668157e+09,7.149642e+09
2010-01-26,4.388480e+09,7.775396e+09,4.979550e+09,9.015405e+09,9.893279e+09,1.453694e+09,2.798905e+09,2.876742e+10,4.276244e+09,1.867720e+10,...,3.170036e+09,2.086400e+09,2.345416e+09,2.433600e+09,4.815297e+09,1.365737e+10,4.143085e+09,3.868293e+09,3.129134e+09,6.998026e+09


In [14]:
price_data

,399551.XSHE,AU1008,FU1308,000602.XSHE,TA0904,TA1103,AL0311,C0605,801121.INDX,150216.XSHE,...,300739.XSHE,399960.XSHE,H30391.XSHG,002109.XSHE,H30418.XSHG,002223.XSHE,399703.XSHE,JM1708,000846.XSHG,002875.XSHE
2005-11-01,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1271.0,959.940,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,822.282,NaN,NaN,NaN
2005-11-02,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1276.0,970.140,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,833.252,NaN,NaN,NaN
2005-11-03,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1276.0,979.060,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,826.929,NaN,NaN,NaN
2005-11-04,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1269.0,979.880,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,831.176,NaN,NaN,NaN
2005-11-07,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1262.0,985.520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,829.739,NaN,NaN,NaN
2005-11-08,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1264.0,1002.450,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,832.781,NaN,NaN,NaN
2005-11-09,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1255.0,991.860,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,828.872,NaN,NaN,NaN
2005-11-10,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1258.0,977.170,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,809.643,NaN,NaN,NaN
2005-11-11,NaN,NaN,NaN,2.6212,NaN,NaN,NaN,1258.0,976.460,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,810.943,NaN,NaN,NaN
2005-11-14,NaN,NaN,NaN,2.5027,NaN,NaN,NaN,1256.0,977.610,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,811.007,NaN,NaN,NaN


In [15]:
df_book_value=df_book_value.fillna(0)
df_market_cap=df_market_cap.fillna(0)
price_data=price_data.fillna(0)

In [16]:
btm = pd.DataFrame(index=index_date,columns=stocks)
i=0
for stk in stocks:
    for date in index_date:
        i=i+1
        if i==1:
            print("type(stk),type(date)",type(stk),stk,type(date),date)
        else:
            pass
#         dt=date.strftime(format)    
#         print("dt",type(dt),dt)
        
        if hasattr(price_data[stk],date):
            if price_data[stk][date]!=0:
                btm[stk][date] = df_book_value[stk][date]/price_data[stk][date]
            else:
                btm[stk][date] = 0
        else:
#             print('hasattr error',stk,date,type(date))
            btm[stk][date]=0
            
btm

type(stk),type(date) <class 'str'> 002334.XSHE <class 'str'> 2010-11-01


,002334.XSHE,000683.XSHE,002326.XSHE,600841.XSHG,000541.XSHE,600520.XSHG,600382.XSHG,600208.XSHG,002168.XSHE,600895.XSHG,...,600422.XSHG,002193.XSHE,600735.XSHG,600179.XSHG,600695.XSHG,600657.XSHG,600531.XSHG,002343.XSHE,000530.XSHE,600491.XSHG
2010-11-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2010-09-30,1.68025,0.500514,0.581385,0.422535,0.230751,0.125381,0.312544,0.32841,0.212407,0.406098,...,0.342293,0.261814,0.185346,0.265385,0.0380711,0.593459,0.803172,0.794324,0.951783,1.04403
2010-08-30,1.64759,0.535614,0.546112,0.390767,0.21723,0.118804,0.334839,0.302479,0.213708,0.402392,...,0.366686,0.26402,0.218146,0.265385,0.0403226,0.546805,0.977524,0.745019,1.01202,0.949858
2010-07-30,2.05418,0.932146,0.68112,0.433603,0.313415,0.120266,0.372122,0.427763,0.261541,0.41427,...,0.39567,0.257852,0.21933,0.298077,0.0440141,0.506169,1.06685,0.785704,1.15391,1.01988
2010-06-30,2.09628,1.2139,0.702604,0.506313,0.35589,0.14373,0.420434,0.496821,0.29155,0.493771,...,0.462329,0.276508,0.230958,0.350679,0.0527426,0.615654,1.25146,0.846427,1.48676,1.35461
2010-05-28,1.75203,0.895089,0.642011,0.426936,0.290418,0.134853,0.334225,0.478752,0.265519,0.443894,...,0.38775,0.259621,0.185393,0.301556,0.0419463,0.565166,1.02933,0.825926,1.06772,1.01596
2010-04-28,2.09944,0.65796,0.663303,0.361112,0.288147,0.131992,0.295205,0.461951,0.232868,0.333929,...,0.349435,0.302989,0.138461,0.283662,0.034965,0.51181,0.846665,0.679719,1.03125,0.885168
2010-03-26,0.702576,0.458546,0.833087,0.372617,0.327478,0.14553,0.276624,0.392067,0.401715,0.351904,...,0.383613,0.264128,0.146276,0.286331,0.0390453,0.409342,0.904473,0.293028,0.92858,0.603796
2010-02-26,0.574126,0.437861,0.29612,0.352225,0.371732,0.158252,0.281664,0.396775,0.407469,0.336949,...,0.355264,0.251227,0.143363,0.276389,0.0407407,0.358597,0.876473,0.265767,1.09074,0.663017
2010-01-26,0.590036,0.453714,0.309379,0.338715,0.408138,0.163297,0.30333,0.398172,0.426552,0.330244,...,0.447865,0.299778,0.171089,0.31891,0.0448034,0.385011,0.904473,0.259301,1.27861,0.677381


In [17]:
##calculate market size medians
median_size = {}
for date in index_date:
    #日市值中位数
    median_size[date] = np.median(df_market_cap.loc[date])

In [24]:
median_size

{'2010-11-01': 4900725398.5,
 '2010-09-30': 4371136000.0,
 '2010-08-30': 4219002000.0,
 '2010-07-30': 3788538500.0,
 '2010-06-30': 3257609263.0,
 '2010-05-28': 3633623914.5,
 '2010-04-28': 4048460000.0,
 '2010-03-26': 3928053550.5,
 '2010-02-26': 3834181430.5,
 '2010-01-26': 3520336698.0,
 '2009-12-25': 3503778480.0,
 '2009-11-25': 3459637353.5,
 '2009-10-23': 2879121500.0,
 '2009-09-23': 2635033795.0,
 '2009-08-21': 2513249992.0,
 '2009-07-21': 2701610251.0,
 '2009-06-19': 2413836000.0,
 '2009-05-19': 2375912700.0,
 '2009-04-17': 2206991228.0,
 '2009-03-17': 1872572164.0,
 '2009-02-17': 1881108473.5,
 '2009-01-16': 1503149700.0,
 '2008-12-16': 1417506800.0,
 '2008-11-14': 1247854997.5,
 '2008-10-14': 1180752187.0,
 '2008-09-12': 1363363842.0,
 '2008-08-12': 1591382880.0,
 '2008-07-11': 1960100000.0,
 '2008-06-11': 2038566400.0,
 '2008-05-09': 2464026989.5,
 '2008-04-09': 2348094000.0,
 '2008-03-07': 3310652120.0,
 '2008-02-05': 2984480104.0,
 '2008-01-04': 3231687850.0,
 '2007-12-04':

In [67]:
##calculate the 30%, 70% quantile of book to market ratio each month

lmark = {}
hmark = {}
for date in index_date:
    lmark[date] = np.percentile(list(btm.loc[date]),30)
    hmark[date] = np.percentile(list(btm.loc[date]),70)
    

In [69]:
lmark

{'2010-11-01': 0.0,
 '2010-09-30': 0.243483954338444,
 '2010-08-30': 0.24431954265877875,
 '2010-07-30': 0.2641095214457449,
 '2010-06-30': 0.2941224070142704,
 '2010-05-28': 0.24803522206194797,
 '2010-04-28': 0.21228587239954663,
 '2010-03-26': 0.19614341571866992,
 '2010-02-26': 0.18579124178857584,
 '2010-01-26': 0.19366549179272574,
 '2009-12-25': 0.1742882627919312,
 '2009-11-25': 0.15824474197927427,
 '2009-10-23': 0.1567200743271594,
 '2009-09-23': 0.16641329317374542,
 '2009-08-21': 0.17169539659367078,
 '2009-07-21': 0.14612215261198264,
 '2009-06-19': 0.16142210270319712,
 '2009-05-19': 0.16804429109204463,
 '2009-04-17': 0.1910752307458,
 '2009-03-17': 0.22097616723729693,
 '2009-02-17': 0.2100217991847612,
 '2009-01-16': 0.2615014444549327,
 '2008-12-16': 0.27626444445904313,
 '2008-11-14': 0.3178581169309992,
 '2008-10-14': 0.31497253292604777,
 '2008-09-12': 0.2797554286139641,
 '2008-08-12': 0.2248929115067017,
 '2008-07-11': 0.17886631118935092,
 '2008-06-11': 0.154341

In [71]:
hmark

{'2010-11-01': 0.0,
 '2010-09-30': 0.6167305022507922,
 '2010-08-30': 0.6113226163916955,
 '2010-07-30': 0.6475038250668125,
 '2010-06-30': 0.7229818031530046,
 '2010-05-28': 0.6293889766996847,
 '2010-04-28': 0.5407657086745425,
 '2010-03-26': 0.47231930601685207,
 '2010-02-26': 0.46283445603548795,
 '2010-01-26': 0.48311516127570675,
 '2009-12-25': 0.4679570243532773,
 '2009-11-25': 0.44603413335026765,
 '2009-10-23': 0.4963194381575523,
 '2009-09-23': 0.5306832958476597,
 '2009-08-21': 0.545823062754996,
 '2009-07-21': 0.5136334470486579,
 '2009-06-19': 0.5665791730133428,
 '2009-05-19': 0.58040287705188,
 '2009-04-17': 0.6218532799211225,
 '2009-03-17': 0.7250553217167914,
 '2009-02-17': 0.7198954824913123,
 '2009-01-16': 0.8917394526960981,
 '2008-12-16': 0.9318501145804474,
 '2008-11-14': 1.035197284276191,
 '2008-10-14': 1.101967665210029,
 '2008-09-12': 0.9641817350027916,
 '2008-08-12': 0.8429325082581174,
 '2008-07-11': 0.6807502232031324,
 '2008-06-11': 0.6433746783418351,
 

In [72]:
for date in index_date:
    if smark[date] == 0:
        smark[date]=hmark[date]/2

In [73]:
smark

{'2010-11-01': 0.0,
 '2010-09-30': 0.243483954338444,
 '2010-08-30': 0.24431954265877875,
 '2010-07-30': 0.2641095214457449,
 '2010-06-30': 0.2941224070142704,
 '2010-05-28': 0.24803522206194797,
 '2010-04-28': 0.21228587239954663,
 '2010-03-26': 0.19614341571866992,
 '2010-02-26': 0.18579124178857584,
 '2010-01-26': 0.19366549179272574,
 '2009-12-25': 0.1742882627919312,
 '2009-11-25': 0.15824474197927427,
 '2009-10-23': 0.1567200743271594,
 '2009-09-23': 0.16641329317374542,
 '2009-08-21': 0.17169539659367078,
 '2009-07-21': 0.14612215261198264,
 '2009-06-19': 0.16142210270319712,
 '2009-05-19': 0.16804429109204463,
 '2009-04-17': 0.1910752307458,
 '2009-03-17': 0.22097616723729693,
 '2009-02-17': 0.2100217991847612,
 '2009-01-16': 0.2615014444549327,
 '2008-12-16': 0.27626444445904313,
 '2008-11-14': 0.3178581169309992,
 '2008-10-14': 0.31497253292604777,
 '2008-09-12': 0.2797554286139641,
 '2008-08-12': 0.2248929115067017,
 '2008-07-11': 0.17886631118935092,
 '2008-06-11': 0.154341

In [92]:
#return of each stock
return_data = pd.DataFrame(index=index_date,columns=stocks)
lastIndex=len(index_date)-1;
for stk in stocks:
    i = 0
    for date in index_date:
        if i+1<=lastIndex and hasattr(price_data[stk],index_date[i+1]):
            if price_data[stk][index_date[i+1]]==0: #倒序的价格，取出昨日价格，即下一行的数据，第二行为0，不能计算
                return_data[stk][date] = 0
            else: 
                if hasattr(price_data[stk],index_date[i]):#第一行有数据
                    return_data[stk][date] = price_data[stk][index_date[i]]/price_data[stk][index_date[i+1]] - 1
                else:
                    #无当天数据，不加这一句的话，打出来为NaN，#2010-11-01 所有数据都是NaN,因为price_date的数据只取到2010-10-29
                    return_data[stk][date]=0
                    print('hasattr error,index_date[i]',stk,date,type(date))
        else:
            return_data[stk][date]=0
            #无上一交易日数据，即第二行，最后一个数据
            print('hasattr error,index_date[i+1]',stk,date,type(date))
        i = i + 1
        

hasattr error,index_date[i] 002334.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002334.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002334.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000683.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000683.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000683.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002326.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002326.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002326.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600841.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600841.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600841.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000541.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000541.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000541.XSHE 2005-09-30 <class 'str'>
hasat

hasattr error,index_date[i+1] 000797.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000797.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600158.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600158.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600158.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600641.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600641.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600641.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600845.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600845.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600845.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600727.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600727.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600727.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 300112.XSHE 2010-11-01 <class 'str'>
hasat

hasattr error,index_date[i+1] 000029.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000029.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000613.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000613.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000613.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600170.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600170.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600170.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000669.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000669.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000669.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600740.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600740.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600740.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000828.XSHE 2010-11-01 <class 'str'>
hasat

hasattr error,index_date[i+1] 002199.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002199.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000065.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000065.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000065.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 601918.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 601918.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 601918.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002166.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002166.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002166.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000895.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000895.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000895.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002088.XSHE 2010-11-01 <class 'str'>
hasat

hasattr error,index_date[i+1] 600639.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600639.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000055.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000055.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000055.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600386.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600386.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600386.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600101.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600101.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600101.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600403.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600403.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600403.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000811.XSHE 2010-11-01 <class 'str'>
hasat

...

hasattr error,index_date[i+1] 600461.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600461.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002023.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002023.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002023.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600425.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600425.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600425.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 300124.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 300124.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 300124.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002149.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002149.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002149.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600234.XSHG 2010-11-01 <class 'str'>
hasat

hasattr error,index_date[i+1] 002090.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002090.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000673.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000673.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000673.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600331.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600331.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600331.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002017.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002017.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002017.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002353.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002353.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002353.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600609.XSHG 2010-11-01 <class 'str'>
hasat

hasattr error,index_date[i+1] 600309.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600309.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000687.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000687.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000687.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 300111.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 300111.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 300111.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 300093.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 300093.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 300093.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000967.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000967.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000967.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600749.XSHG 2010-11-01 <class 'str'>
hasat

hasattr error,index_date[i+1] 600021.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600021.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600622.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600622.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600622.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600125.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600125.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600125.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002459.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002459.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002459.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 002024.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 002024.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 002024.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600446.XSHG 2010-11-01 <class 'str'>
hasat

hasattr error,index_date[i+1] 600361.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600361.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600019.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600019.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600019.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 000676.XSHE 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 000676.XSHE 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 000676.XSHE 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600707.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600707.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600707.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 600820.XSHG 2010-11-01 <class 'str'>
hasattr error,index_date[i+1] 600820.XSHG 2005-11-04 <class 'str'>
hasattr error,index_date[i+1] 600820.XSHG 2005-09-30 <class 'str'>
hasattr error,index_date[i] 601718.XSHG 2010-11-01 <class 'str'>
hasat

In [93]:
return_data

,002334.XSHE,000683.XSHE,002326.XSHE,600841.XSHG,000541.XSHE,600520.XSHG,600382.XSHG,600208.XSHG,002168.XSHE,600895.XSHG,...,600422.XSHG,002193.XSHE,600735.XSHG,600179.XSHG,600695.XSHG,600657.XSHG,600531.XSHG,002343.XSHE,000530.XSHE,600491.XSHG
2010-11-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2010-09-30,-0.0194411,0.0701295,-0.0606697,-0.0751858,-0.0585932,-0.0524632,0.0713324,-0.0789606,0.00612641,-0.00912529,...,0.0712653,0.00842379,0.176966,0,-0.0467742,-0.078613,0.21708,-0.062071,0.0632877,-0.0901994
2010-08-30,0.285035,0.163114,0.216795,0.104086,0.353589,0.0123057,0.112182,-0.0397176,0.275579,-0.0436456,...,0.0821381,-0.0209353,0.103311,0,0.0915493,-0.0657287,0.0913772,0.0546094,0.133196,0.0980875
2010-07-30,0.020495,0.302264,0.0315422,0.167689,0.135523,0.210031,0.129829,0.16144,0.114741,0.191907,...,0.168472,0.0723519,0.0530155,0.176471,0.198312,0.2163,0.13519,0.109489,0.288456,0.328209
2010-06-30,-0.164219,-0.262634,-0.0862408,-0.156774,-0.183967,-0.0617647,-0.205046,-0.0363697,-0.0892849,-0.101014,...,-0.161311,-0.0610722,-0.197286,-0.140078,-0.204698,-0.0820064,-0.177499,-0.0242203,-0.281851,-0.25
2010-05-28,0.19829,-0.259382,0.0331651,-0.154179,0.0335189,-0.145192,-0.11675,-0.0350934,-0.122971,-0.2075,...,-0.0988126,0.167043,-0.174147,-0.0771993,-0.166434,-0.0891266,-0.177458,-0.177022,-0.0341497,-0.112398
2010-04-28,0.15081,0.118445,0.273438,0.0475346,0.136499,0.102564,-0.0522535,-0.164751,0.0685164,0.05383,...,0.151099,-0.0898653,0.0564418,-0.198561,-0.124847,-0.200208,0.148763,-0.0528567,-0.0958577,-0.286665
2010-03-26,-0.00652796,-0.0451103,0.0266772,-0.0326858,0.135133,0.0874152,0.106578,0.0120082,0.0917214,0.00591554,...,-0.0738984,-0.048846,-0.019914,-0.0347222,0.0434227,-0.00831992,-0.0309567,-0.0401571,0.17463,0.0980803
2010-02-26,0.0277128,0.0362056,0.0447772,-0.0383556,0.0979377,0.0318818,0.0769231,0.0776435,0.0468351,-0.0198981,...,0.260653,0.193256,0.193397,0.153846,0.0997191,0.0736604,0.0319456,-0.0243313,0.172244,0.0216644
2010-01-26,0,-0.0559422,0.158392,0.00535495,-0.0241386,-0.172458,-0.0421069,-0.0957408,0.0948666,-0.0226922,...,-0.0648749,0.109788,0.141673,-0.074184,-0.0246575,-0.21334,-0.128245,0,-0.0529463,-0.144767


In [95]:
##calculate SMB and HMI
smb = pd.Series(index = index_date)
hmi = pd.Series(index = index_date)
first_row = True
for date in index_date:
    if first_row:
        smb[date]=0
        hmi[date]=0
        first_row=False
        continue
    small_size=0.0
    big_size=0.0
    value_btm=0.0
    middle_btm=0.0
    growth_btm=0.0
    for stk in stocks:
        #计算市值size分组
        if df_market_cap[stk][date]<median_size[date]:
            small_size = small_size + return_data[stk][date]*df_market_cap[stk][date]#计算每日总的回报＝回报率＊市值
        else: 
            big_size = big_size + return_data[stk][date]*df_market_cap[stk][date]
        #计算btm分组
        if btm[stk][date]<smark[date]:
            growth_btm = growth_btm + return_data[stk][date]*df_market_cap[stk][date]
        elif btm[stk][date]>hmark[date]:
            value_btm = value_btm + return_data[stk][date]*df_market_cap[stk][date]
        else:
            middle_btm = middle_btm + return_data[stk][date]*df_market_cap[stk][date]

    mktcap = np.sum(df_market_cap.loc[date])
    smb[date] = (small_size - big_size)/mktcap#每日回报率＝每日总回报/总市值
    hmi[date] = (value_btm - growth_btm)/mktcap

In [96]:
smb

2010-11-01    0.000000
2010-09-30   -0.013197
2010-08-30   -0.010967
2010-07-30   -0.088186
2010-06-30    0.068859
2010-05-28    0.053741
2010-04-28    0.033689
2010-03-26   -0.003568
2010-02-26   -0.028652
2010-01-26    0.030648
2009-12-25    0.034655
2009-11-25   -0.055501
2009-10-23   -0.090743
2009-09-23    0.021354
2009-08-21    0.060012
2009-07-21   -0.123522
2009-06-19   -0.114692
2009-05-19   -0.067433
2009-04-17   -0.128752
2009-03-17    0.032843
2009-02-17   -0.181596
2009-01-16    0.009471
2008-12-16   -0.005451
2008-11-14   -0.002004
2008-10-14    0.019034
2008-09-12    0.134118
2008-08-12    0.113768
2008-07-11    0.035417
2008-06-11    0.145264
2008-05-09   -0.069974
2008-04-09    0.155272
2008-03-07    0.055782
2008-02-05    0.121146
2008-01-04   -0.099963
2007-12-04    0.081414
2007-11-02   -0.090193
2007-09-28   -0.092456
2007-08-28   -0.226177
2007-07-27   -0.069967
2007-06-27   -0.030703
2007-05-25   -0.114603
2007-04-25   -0.232478
2007-03-23   -0.033536
2007-02-16 

In [97]:
hmi

2010-11-01    0.000000
2010-09-30   -0.027027
2010-08-30   -0.017427
2010-07-30    0.028008
2010-06-30   -0.024543
2010-05-28   -0.018548
2010-04-28   -0.036493
2010-03-26    0.002683
2010-02-26   -0.002521
2010-01-26   -0.015912
2009-12-25   -0.011978
2009-11-25    0.016753
2009-10-23    0.034063
2009-09-23   -0.021985
2009-08-21   -0.023113
2009-07-21    0.034151
2009-06-19    0.019351
2009-05-19    0.026123
2009-04-17    0.037531
2009-03-17   -0.008656
2009-02-17    0.046352
2009-01-16    0.007210
2008-12-16    0.007199
2008-11-14    0.006408
2008-10-14   -0.010760
2008-09-12   -0.013418
2008-08-12   -0.023642
2008-07-11   -0.006847
2008-06-11   -0.028554
2008-05-09    0.007009
2008-04-09   -0.067315
2008-03-07   -0.020008
2008-02-05   -0.022037
2008-01-04    0.015329
2007-12-04   -0.010448
2007-11-02   -0.013006
2007-09-28    0.003019
2007-08-28    0.031810
2007-07-27    0.016099
2007-06-27   -0.032344
2007-05-25    0.016503
2007-04-25    0.041719
2007-03-23    0.013490
2007-02-16 

In [109]:
#market (HuShen300)
# df_Rm_data = get_price(['000300.XSHG'],start_date='2005-11-01', end_date='2010-11-05',frequency='1d',fields='ClosingPx')
df_Rm_data=pd.read_csv("df_Rm_data.csv",index_col=0,header=None)
df_Rm_data

,1
0,
2005-09-30,917.392
2005-10-10,907.321
2005-10-11,919.720
2005-10-12,923.485
2005-10-13,916.499
2005-10-14,904.828
2005-10-17,897.623
2005-10-18,902.366
2005-10-19,898.744


In [110]:
#market return(HuShen300) and risk free return(0.375724091% 1M, for I can't get this rate from Ricequant nor can I upload my own data)
Rm = pd.Series(index = index_date)
Rf = pd.Series(0.00375724091,index = index_date)

for i in range(61):
    if i==60:
        Rm[index_date[i]] = 0
        Rf[index_date[i]] = 0
        continue
    Rm[index_date[i]] = df_Rm_data.loc[index_date[i]]/df_Rm_data.loc[index_date[i+1]] - 1
    

IndexError: index 60 is out of bounds for axis 0 with size 60